# Hacker News Posts Analysis

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

We can find the data set [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts).

**Our goal with this project is to analyze the best times for posting to get maximum reach.**

Below are descriptions of the columns:

`id`: the unique identifier from Hacker News for the post
`title`: the title of the post
`url`: the URL that the posts links to, if the post has a URL
`num_points`: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
`num_comments`: the number of comments on the post
`author`: the username of the person who submitted the post
`created_at`: the date and time of the post's submission

# Data Exploration

We will begin by importing the data using `csv` module and analysing it. 

In [1]:
import csv

opened_file = open("C:/Users/Linus/Documents/Sheets/HN_posts_year_to_Sep_26_2016.csv", encoding='utf-8')
read_file = csv.reader(opened_file)
hn = list(read_file)

print('Total number of posts :', len(hn))
print(hn[:5])

Total number of posts : 293120
[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']]


In [2]:
header = hn[0]   # separating header
hn = hn[1:]      
print(hn)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Extracting Ask HN and Show HN Posts

Now that we've removed the headers from `hn`, we're ready to filter our data. Since we're only concerned with post titles beginning with `Ask HN` or `Show HN`, we'll create new lists of lists containing just the data for those titles.

To find the posts that begin with either `Ask HN` or `Show HN`, we'll use the string method `startswith`. Given a string object, say, `string1`, we can check if starts with, say, `dq` by inspecting the output of the object `string1.startswith('dq')`. If `string1` starts with `dq`, it will return `True`; otherwise, it will return `False`.

However, it's case-sensistive. For example,`'dataquest'.startswith('Data')` returns `False`. To take care of cases, we will use `lower` method.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

        
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    if title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
total_ask_posts = len(ask_posts)
total_show_posts = len(show_posts)

print('Number of ask posts:', len(ask_posts))
print(ask_posts[:3])
print('\n')
print('Number of show posts:', len(show_posts))
print(show_posts[:3])

Number of ask posts: 9139
[['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'], ['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17'], ['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57']]


Number of show posts: 10158
[['12578335', 'Show HN: Finding puns computationally', 'http://puns.samueltaylor.org/', '2', '0', 'saamm', '9/26/2016 0:36'], ['12578182', 'Show HN: A simple library for complicated animations', 'https://christinecha.github.io/choreographer-js/', '1', '0', 'christinecha', '9/26/2016 0:01'], ['12578098', 'Show HN: WebGL visualization of DNA sequences', 'http://grondilu.github.io/dna.html', '1', '0', 'grondilu', '9/25/2016 23:44']]


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In the previous cell, we discovered that there are 9139 "ask posts" and 10158 "show posts". It seems like "show posts" are more popular. But let's analyse further to see if its true. 

We will now calcuate the avearge number of comments recieved by post for both Ask HN and Show HN posts. 

We will calculate average by adding all the comments in `ask_posts` list and dividing it by number of posts. And repeat the same for `show_posts`.

In [4]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments/total_ask_posts
print('Average comments for Ask HN posts:', avg_ask_comments)

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments/total_show_posts
print('Average comments for Show HN posts:', avg_show_comments)

Average comments for Ask HN posts: 10.393478498741656
Average comments for Show HN posts: 4.886099625910612


As we can see, `Ask HN` posts get more  comments than `Show HN` posts. That is quite obvious given people are more likely to answer questions. But now we also have statistical proof. 

# Finding the Number of Ask Posts and Comments by Hour Created

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We'll use the `datetime` module to work with the data in the `created_at` column. We can use the `datetime.strptime()` constructor to parse dates stored as strings and return datetime objects:

In [5]:
import datetime as dt

result_list = []
for row in ask_posts:
    result_list.append([row[6], int(row[4])])

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time = dt.datetime.strptime(row[0], '%m/%d/%Y %H:%M')
    
    if time.hour not in counts_by_hour:
        counts_by_hour[time.hour] = 1
        comments_by_hour[time.hour] = row[1]
    else:
        counts_by_hour[time.hour] += 1
        comments_by_hour[time.hour] += row[1]
        
print('Counts by hour:', counts_by_hour)
print('\n')
print('Comments by hour:', comments_by_hour)

Counts by hour: {2: 269, 1: 282, 22: 383, 21: 518, 19: 552, 17: 587, 15: 646, 14: 513, 13: 444, 11: 312, 10: 282, 9: 222, 7: 226, 3: 271, 23: 343, 20: 510, 16: 579, 8: 257, 0: 301, 18: 614, 12: 342, 4: 243, 6: 234, 5: 209}


Comments by hour: {2: 2996, 1: 2089, 22: 3372, 21: 4500, 19: 3954, 17: 5547, 15: 18525, 14: 4972, 13: 7245, 11: 2797, 10: 3013, 9: 1477, 7: 1585, 3: 2154, 23: 2297, 20: 4462, 16: 4466, 8: 2362, 0: 2277, 18: 4877, 12: 4234, 4: 2360, 6: 1587, 5: 1838}


# Calculating the Average Number of Comments for Ask HN Posts by Hour

We created two dictionaries:

1. `counts_by_hour`: contains the number of ask posts created during each hour of the day.
2. `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received.

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [6]:
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])
    
print(avg_by_hour)

[[2, 11.137546468401487], [1, 7.407801418439717], [22, 8.804177545691905], [21, 8.687258687258687], [19, 7.163043478260869], [17, 9.449744463373083], [15, 28.676470588235293], [14, 9.692007797270955], [13, 16.31756756756757], [11, 8.96474358974359], [10, 10.684397163120567], [9, 6.653153153153153], [7, 7.013274336283186], [3, 7.948339483394834], [23, 6.696793002915452], [20, 8.749019607843136], [16, 7.713298791018998], [8, 9.190661478599221], [0, 7.5647840531561465], [18, 7.94299674267101], [12, 12.380116959064328], [4, 9.7119341563786], [6, 6.782051282051282], [5, 8.794258373205741]]


# Sorting and Printing Values from a List of Lists

We've now calculated the average number of comments for posts created during each hour of the day, and stored the results in a list of lists named `avg_by_hour`. 

Although we now have the results we need, this format makes it difficult to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [7]:
swap_avg_by_hour = []

for hour in avg_by_hour:
    swap_avg_by_hour.append([hour[1], hour[0]]) 
    
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print(sorted_swap)

[[28.676470588235293, 15], [16.31756756756757, 13], [12.380116959064328, 12], [11.137546468401487, 2], [10.684397163120567, 10], [9.7119341563786, 4], [9.692007797270955, 14], [9.449744463373083, 17], [9.190661478599221, 8], [8.96474358974359, 11], [8.804177545691905, 22], [8.794258373205741, 5], [8.749019607843136, 20], [8.687258687258687, 21], [7.948339483394834, 3], [7.94299674267101, 18], [7.713298791018998, 16], [7.5647840531561465, 0], [7.407801418439717, 1], [7.163043478260869, 19], [7.013274336283186, 7], [6.782051282051282, 6], [6.696793002915452, 23], [6.653153153153153, 9]]


# Top 5 Hours for Ask Posts Comments:


In [8]:
print("Top 5 Hours for Ask Posts Comments")
template = "{time}: {comments:.2f} average comments per post"

for hour in sorted_swap[:5]:
    post_time = str(hour[1])
    formatted_time = dt.datetime.strptime(post_time, "%H")
    str_time = formatted_time.strftime("%H:%M")
    avg_comments = hour[0]
    post = template.format(time=str_time, comments=avg_comments)
    print(post)

Top 5 Hours for Ask Posts Comments
15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post
10:00: 10.68 average comments per post


# Accounting for Timezone difference 

Our analysis concludes that afternoon and late morning are good time to ask your questions. Though 2AM is an interesting anamoly. 

As per the [dataset](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts), the time zone of posts made is Eastern Timezone (EST/UTC-5). 

If you're in Indian timezone (IST/UTC+5:30), there is 10.30 hours difference between both times (or 9.30 hours when Daylight Saving Time is observed). 